In [ ]:
# this is the final code

import nltk
import spacy
import os
import random

# Download tokenizer
nltk.download('punkt')

# Load spaCy model (prefer large for best similarity)
nlp = spacy.load("en_core_web_lg")


# === Intent responses ===
intent_responses = {
    "greeting": ["Hello!", "Hi there!", "Hey! How can I assist you?"],
    "goodbye": ["Goodbye!", "See you later!", "Bye! Have a great day!"],
    "name": ["I'm your friendly Bayern Munich chatbot!", "I go by BayernBot!"],
    "feeling": ["I'm just code, but I feel awesome!", "All systems operational!"],
    "thanks": ["You're welcome!", "No problem!", "Glad I could help!"],
    "help": ["You can ask me about Bayern Munich’s history, trophies, players, stadium, or anything else!"],
    "creator": ["I was created by a curious developer to answer your Bayern Munich questions."],
    "club_type": ["Bayern Munich is a professional football club based in Munich, Germany."],
    "colors": ["Bayern's traditional team colors are red and white."]
}

# === Intent keywords ===
intent_keywords = {
    "greeting": ["hello", "hi", "hey", "what's up", "sup"],
    "goodbye": ["bye", "goodbye", "see you", "talk to you later","quit","exit"],
    "name": ["your name", "who are you", "what should I call you"],
    "feeling": ["how are you", "how do you feel", "what's up"],
    "thanks": ["thank you", "thanks", "appreciate it", "thx"],
    "help": ["help", "what can you do", "how can you help"],
    "creator": ["who created you", "who made you"],
    "club_type": ["what is bayern", "what kind of club", "type of club"],
    "colors": ["team colors", "jersey", "kit colors"],
    "joke": ["tell me a joke", "joke", "make me laugh", "say something funny"]
}

# === Joke generator ===
def get_random_joke():
    jokes = [
        "Why don't scientists trust atoms? Because they make up everything!",
        "Why did the math book look sad? Because it had too many problems.",
        "I told my computer I needed a break, and it said 'No problem — I'll crash.'",
        "Why don’t programmers like nature? It has too many bugs.",
        "What’s a computer’s favorite snack? Microchips!"
    ]
    return random.choice(jokes)

# === Classify basic intent ===
def classify_intent(user_input):
    user_input = user_input.lower()
    for intent, keywords in intent_keywords.items():
        if any(keyword in user_input for keyword in keywords):
            return intent
    return None

# === Load Bayern text into sentences ===
def load_sentences(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]

# CHANGE THIS PATH to your local file path if needed
sentences = load_sentences(r"bayern.txt")

# === Direct keyword-based answers ===
direct_answers = {
    "founder": "Established in 1900 by eleven players led by Franz John, Bayern won its first national title in 1932.",
    "founded": "Established in 1900 by eleven players led by Franz John, Bayern won its first national title in 1932.",
    "when started": "Established in 1900 by eleven players led by Franz John, Bayern won its first national title in 1932.",
    "created": "Established in 1900 by eleven players led by Franz John, Bayern won its first national title in 1932.",
    "sextuple": "With their 2020 FIFA Club World Cup victory, Bayern became only the second club to achieve the 'sextuple'—winning six major trophies across one season and the following calendar year."
}

# === Semantic similarity matching ===
def find_best_sentence(user_input):
    user_doc = nlp(user_input)
    best_score = 0.0
    best_sentence = None

    for sent in sentences:
        score = user_doc.similarity(nlp(sent))
        if score > best_score:
            best_score = score
            best_sentence = sent

    return best_sentence if best_score > 0.65 else None

# === Respond function ===
def respond(user_input):
    intent = classify_intent(user_input)
    
    if intent:
        if intent == "joke":
            return get_random_joke()
        return random.choice(intent_responses[intent])

    for keyword in direct_answers:
        if keyword in user_input.lower():
            return direct_answers[keyword]

    match = find_best_sentence(user_input)
    if match:
        return match

    return "I'm not sure how to answer that. Try asking something else about Bayern Munich."

# === Chatbot loop ===
def chatbot():
    print("ChatBot: Hello! Ask me anything about Bayern Munich, or just chat. Type 'bye' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.strip() == "":
            continue
        response = respond(user_input)
        print("ChatBot:", response)
        if user_input.lower() in ["bye", "exit", "quit"]:
            break

if __name__ == "__main__":
    chatbot()
